In [1]:
from get_data import get_data
import torch
import os
import numpy as np

from datasets import InMemDataLoader


#### create list of all nifti file paths ###
#d ='/bigdata/hplsim/aipp/RLtract/deepFibreTracking/examples/data/HCP_extended/'
'''d = '/bigdata/hplsim/aipp/RLtract/deepFibreTracking/examples/data/HCP_extended/'
all_paths = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
all_paths = [p + '/T1w/T1w_acpc_dc_restore_1.25.nii.gz' for p in all_paths]'''

'''d = '/home/ramana44/all_scans_single_channel_equal_dim/'
all_paths = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
all_paths = [p for p in all_paths]'''
all_paths = []
for root, dirs, files in os.walk(os.path.abspath("/home/ramana44/all_scans_single_channel_equal_dim/")):
    for file in files:
        #print(os.path.join(root, file))
        all_paths.append((os.path.join(root, file)))

print('len(all_paths)',len(all_paths))


### load data ###
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_train_test_set(paths, device, batch_size=200, train_set_size=0.8, test_set_size=0.2):
    assert train_set_size + test_set_size <= 1., "Train and test set size should not exceed 100%"
    
    path_indices = np.arange(len(paths))
    #np.random.shuffle(path_indices)                             # randomize indices of the paths for train and test set selection
    
    num_train = int(np.round_(len(paths) * train_set_size))     # calc amount of training sets to load
    num_test = int(np.round_(len(paths) * test_set_size))       # calc amount of test sets to load
    train_indices = path_indices[:num_train]                    # select unique and random indices from all paths
    test_indices = path_indices[-num_test:]                     # for train and test set


    train_data = get_data([paths[i] for i in train_indices], device)  # only load specific indices preveiously selected
    test_data = get_data([paths[i] for i in test_indices], device)
    print('train_data.shape',train_data.shape)

    train_loader = InMemDataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True) # init dataloader for train and test set
    test_loader = InMemDataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last=True) 
    return train_loader, test_loader



reg_nodes_sampling = "legendre"
alpha = 0.5
hidden_size = 1000
deg_poly = 20
latent_dim = 80
lr = 1e-4
no_layers = 5
train_set_size = 0.8



len(all_paths) 2451


In [4]:
train_loader, test_loader = get_train_test_set(all_paths, device, train_set_size=train_set_size, batch_size=200)

before normalize 1961
after normalize 1961
initial torch.Size([1961, 96, 96, 60])
after permute torch.Size([1961, 60, 96, 96])
after 123 torch.Size([1961, 25, 96, 96])
after reshape torch.Size([49025, 96, 96])
after unsqueeze torch.Size([49025, 1, 96, 96])
before normalize 490
after normalize 490
initial torch.Size([490, 96, 96, 60])
after permute torch.Size([490, 60, 96, 96])
after 123 torch.Size([490, 25, 96, 96])
after reshape torch.Size([12250, 96, 96])
after unsqueeze torch.Size([12250, 1, 96, 96])
train_data.shape torch.Size([49025, 1, 96, 96])


In [5]:
batch_x = next(iter(train_loader))